# Lab 16 Assignment 3 - Group Assignment

When creating ML models, the concept of efficiency has three sides:
1. The time dedicated by the analyst to build the model
2. The computer time and resources needed by the final model
3. The accuracy of the final model

Efficiency is a combination of all

In this assignment, you are asked to be efficient. Spark is the best tool to build models over massive datasets

If you need to create Spark+Python Machine Learning models that "run fast" on the  cluster, you must avoid using Python code or working with RRD+python. Try to use  the already existing methods that do what you need (do not reinvent the wheel).

Therefore try to use the implemented object+methods inside the Spark SQL and ML modules. They are very fast, because it is compiled Java/Scala code. Try to use: DataFrames, Feature Transfomers, Estimators, Pipelines, GridSearch, CV, ...

For this assignment, you are asked to create a classification model that:
1. Uses the variables in the dataset (train.csv) to predict label "loan_status"
2. Write a python scripts that:
    - Reads the "train.csv" and "test.csv" files, transform and select variables as you wish.
    - Train/fit your model using the "train.csv".
    - Predict your model on the "test.csv" ( you should generate a file with your predictions).
    - I will use a different test dataset (with the true loan_status).

Your work will be evaluated under the following scoring schema
- (40%) ETL process
- (40%) Model train process
- (10%) Code Readability 
- (10%) AUC on the test set (at least 50%)

Enjoy it and best of luck!!


This Assignment is based on kaggle competition https://www.kaggle.com/c/loan-default-prediction from where a sub-dataset has been taken.

### File descriptions
**train.csv** - the training set (to use for building a model)

**test.csv** - the test set (to use for applying predictings)

**sample_submission.csv** - a template for the submission file

### Data Description (also contained in LendingClub_DataDescription.csv)
**ID**: A unique LC assigned ID for the loan listing.

**loan_amnt**: The listed amount of the loan applied for by the borrower. If at some point in time, the credit department reduces the loan amount, then it will be reflected in this value.

**loan_status**: Current status of the loan (**Target**: 1 = Charged Off, 0 = Fully Paid).

**term**: The number of payments on the loan. Values are in months and can be either 36 or 60.

**int_rate**: Interest Rate on the loan.

**installment**: The monthly payment owed by the borrower if the loan originates.

**emp_length**: Employment length in years. Possible values are between 0 and 10 where 0 means less than one year and 10 means ten or more years.

**home_ownership**: The home ownership status provided by the borrower during registration. Our values are: OTHER/NONE, MORTGAGE, OWN, RENT.

**annual_inc**: The self-reported annual income provided by the borrower during registration.

**purpose**: A category provided by the borrower for the loan request.

**title**: The loan title provided by the borrower.

**STATE**: The state provided by the borrower in the loan application.

**delinq_2yrs**: The number of 30+ days past-due incidences of delinquency in the borrower's credit file for the past 2 years.

**revol_bal**: Total credit revolving balance.

**revol_util**: Revolving line utilization rate, or the amount of credit the borrower is using relative to all available revolving credit.

**total_pymnt**: Indicates total payment at the end of the loan.

In [1]:
import os
import sys

os.environ['SPARK_HOME'] = "C:\\Spark\\spark-2.3.2-bin-hadoop2.7\\"

# Create a variable for our root path
SPARK_HOME = os.environ['SPARK_HOME']

#Add the following paths to the system path. Please check your installation
#to make sure that these zip files actually exist. The names might change
#as versions change.
sys.path.insert(0,os.path.join(SPARK_HOME,"python"))
sys.path.insert(0,os.path.join(SPARK_HOME,"python","lib"))
sys.path.insert(0,os.path.join(SPARK_HOME,"python","lib","pyspark.zip"))
sys.path.insert(0,os.path.join(SPARK_HOME,"python","lib","py4j-0.10.7-src.zip"))

#Initialize SparkSession and SparkContext
from pyspark.sql import SparkSession

#Create a Spark Session
spark = SparkSession \
    .builder \
    .master("local[2]") \
    .appName("MiPrimer") \
    .config("spark.executor.memory", "6g") \
    .config("spark.cores.max","4") \
    .getOrCreate()


#Get the Spark Context from Spark Session    
sc = spark.sparkContext

In [2]:
from pyspark.sql.functions import isnan, when, count, col, isnull, avg
from pyspark.mllib.stat import Statistics
from pyspark.sql import functions as f
from pyspark.sql.types import IntegerType
import pandas as pd
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.feature import OneHotEncoderEstimator


def numerical_features(df):
    columnList = [var[0] for var in df.dtypes if (var[1].startswith('int') | var[1].startswith('double'))]
    return columnList

def categorical_features(df):
    columnList = [var[0] for var in df.dtypes if (var[1].startswith('string'))]
    return columnList

def string_replace(df, column, inputs, category_name):
    regex = "/" + "|".join(inputs) + "/i"
    df = df.withColumn(column, f.regexp_replace(column,regex, category_name))
    return df

### 1. Read the files into a Dataframes and show them

In [3]:
loanDF =  spark.read.format('csv') \
            .option("inferSchema", "true") \
            .option("delimiter", ";") \
            .option('header','true') \
            .load('../data/train.csv')

testDF = spark.read.format('csv') \
            .option("inferSchema", "true") \
            .option("delimiter", ";") \
            .option('header','true') \
            .load('../data/test.csv')

loanDF.limit(10).toPandas()

,ID,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,purpose,title,STATE,delinq_2yrs,revol_bal,revol_util,total_pymnt,loan_status
0,2,2500,60 months,15.27%,59.83,< 1 year,RENT,30000.0,car,bike,GA,0,1687,0.094,1014.530000,1
1,4,10000,36 months,13.49%,339.31,10+ years,RENT,49200.0,other,personel,CA,0,5598,0.21,12231.890000,0
2,5,3000,60 months,12.69%,67.79,1 year,RENT,80000.0,other,Personal,OR,0,27783,0.539,4066.908161,0
3,6,5000,36 months,7.90%,156.46,3 years,RENT,36000.0,wedding,My wedding loan I promise to pay back,AZ,0,7963,0.283,5632.210000,0
4,7,7000,60 months,15.96%,170.08,8 years,RENT,47004.0,debt_consolidation,Loan,NC,0,17726,0.856,10137.840010,0
5,8,3000,36 months,18.64%,109.43,9 years,RENT,48000.0,car,Car Downpayment,CA,0,8221,0.875,3939.135294,0
6,10,5375,60 months,12.69%,121.45,< 1 year,RENT,15000.0,other,Building my credit history.,TX,0,9279,0.365,1484.590000,1
7,11,6500,60 months,14.65%,153.45,5 years,OWN,72000.0,debt_consolidation,High intrest Consolidation,AZ,0,4032,0.206,7678.017673,0
8,12,12000,36 months,12.69%,402.54,10+ years,OWN,75000.0,debt_consolidation,Consolidation,CA,0,23336,0.671,13947.989160,0
9,14,3000,36 months,9.91%,96.68,3 years,RENT,15000.0,credit_card,citicard fund,IL,0,7323,0.431,3480.269999,0


In [4]:
testDF.limit(10).toPandas()

,ID,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,purpose,title,STATE,delinq_2yrs,revol_bal,revol_util,total_pymnt,loan_status
0,1,5000,36 months,10.65%,162.87,10+ years,RENT,24000.0,credit_card,Computer,AZ,0,13648,0.84,5863.155187,1
1,3,2400,36 months,15.96%,84.33,10+ years,RENT,12252.0,small_business,real estate business,IL,0,2956,0.99,3005.666844,1
2,9,5600,60 months,21.28%,152.39,4 years,OWN,40000.0,small_business,Expand Business & Buy Debt Portfolio,CA,0,5210,0.33,647.500000,1
3,13,9000,36 months,13.49%,305.38,< 1 year,RENT,30000.0,debt_consolidation,freedom,VA,0,10452,0.92,2277.320000,1
4,15,10000,36 months,10.65%,325.74,3 years,RENT,100000.0,other,Other Loan,CA,0,11997,0.56,7471.990000,1
5,18,3600,36 months,6.03%,109.57,10+ years,MORTGAGE,110000.0,major_purchase,Holiday,CT,0,22836,0.16,3785.271965,1
6,26,15000,36 months,9.91%,483.38,2 years,MORTGAGE,92000.0,credit_card,No more credit card debt!,IL,0,13707,0.94,15823.999050,1
7,27,15000,36 months,14.27%,514.64,9 years,RENT,60000.0,debt_consolidation,consolidation,NY,0,5872,0.58,0.000000,1
8,29,4000,36 months,11.71%,132.31,10+ years,MORTGAGE,106000.0,debt_consolidation,Debt Consolidation,FL,1,6110,0.38,4486.293519,1
9,31,4375,36 months,7.51%,136.11,7 years,MORTGAGE,17108.0,debt_consolidation,Debt Consolidation,NY,0,11210,0.87,4899.960000,1


In [5]:
loanDF.describe().toPandas()

,summary,ID,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,purpose,title,STATE,delinq_2yrs,revol_bal,revol_util,total_pymnt,loan_status
0,count,29755,29755,29755,29755,29755,29755,29755,29755,28526,28517,28516,28516,28516,28480,28516,29755
1,mean,19942.09615190724,11218.509494202655,None,None,324.23834481599505,None,None,69044.8235268022,None,117554.32432432432,None,0.14595314910927198,13350.529071398512,0.5008419840184288,12143.791490200982,0.14199294236262813
2,stddev,11462.221444945904,7431.662873498601,None,None,207.83497941026226,None,None,66683.2408043667,None,673363.3478144147,None,0.4856901246589267,15948.46067741967,0.27686595987774415,9085.146711349636,0.3490487663481529
3,min,2,500,36 months,10.00%,15.69,1 year,MORTGAGE,4000.0,car,\tdebt_consolidation,AK,0,0,-,0.0,0
4,max,39759,35000,60 months,9.99%,1305.19,n/a,RENT,6000000.0,wedding,îîMY FIRST CAR îî,WY,8,149588,0.999,58886.47343,1


In [6]:
testDF.describe().toPandas()

,summary,ID,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,purpose,title,STATE,delinq_2yrs,revol_bal,revol_util,total_pymnt,loan_status
0,count,10024,10024,10024,10024,10024,9738,10024,10024,9632,9632,9630,9630,9630,9618,9630,10024
1,mean,19735.35903830806,11271.71288906624,None,None,326.2570470869901,None,None,68784.71966081405,None,673362.3,None,0.14922118380062305,13492.502076843199,0.4996844349680202,12180.201509834418,1.0
2,stddev,11545.038922347956,7562.425315839945,None,None,212.15251156256915,None,None,54198.150845666925,None,2123579.2154532303,None,0.5061229784339346,15848.321061950159,0.27435058698906706,9275.927785309726,0.0
3,min,1,500,36 months,10.00%,16.25,1 year,MORTGAGE,4800.0,car,Business Loan,AK,0,0,-,0.0,1
4,max,39779,35000,60 months,9.99%,1288.1,< 1 year,RENT,1782000.0,wedding,zandercade,WY,11,149000,1,56849.26986,1


In [7]:
categoricals = categorical_features(loanDF)
categoricals

['term',
 'int_rate',
 'emp_length',
 'home_ownership',
 'purpose',
 'title',
 'STATE',
 'revol_util']

In [8]:
nums = numerical_features(loanDF)
nums

['ID',
 'loan_amnt',
 'installment',
 'annual_inc',
 'delinq_2yrs',
 'revol_bal',
 'total_pymnt',
 'loan_status']

In [9]:
loanDF.toPandas()['STATE'].unique()

array(['GA', 'CA', 'OR', 'AZ', 'NC', 'TX', 'IL', 'MO', 'UT', 'FL', 'PA',
       'MN', 'NJ', None, 'OH', 'SC', 'MA', 'NY', 'CT', 'WA', 'WI', 'VA',
       'AL', 'CO', 'KS', 'LA', 'NV', 'WV', 'MD', 'VT', 'MI', 'DC', 'SD',
       'NH', 'RI', 'AR', 'NM', 'MT', 'HI', 'WY', 'KY', 'AK', 'OK', 'DE',
       'TN', 'IA', 'MS', 'NE', 'ID', 'IN', 'ME'], dtype=object)

In [10]:
loanDF.toPandas()['term'].unique()

array([' 60 months', ' 36 months'], dtype=object)

In [11]:
loanDF.toPandas()['emp_length'].unique()

array(['< 1 year', '10+ years', '1 year', '3 years', '8 years', '9 years',
       '5 years', '4 years', '6 years', '2 years', '7 years', 'n/a'],
      dtype=object)

In [12]:
loanDF.toPandas()['home_ownership'].unique()

array(['RENT', 'OWN', 'MORTGAGE', 'OTHER', 'NONE'], dtype=object)

In [13]:
loanDF.toPandas()['purpose'].unique()

array(['car', 'other', 'wedding', 'debt_consolidation', 'credit_card',
       'home_improvement', 'medical', 'major_purchase', None,
       'small_business', 'vacation', 'moving', 'house',
       'renewable_energy', 'educational'], dtype=object)

In [14]:
loanDF.toPandas()['title'].unique()

array(['bike', 'personel', 'Personal', ..., 'New kitchen',
       'Mortgage Pre-Payment', 'Recycling Plant in East Africa'],
      dtype=object)

In [15]:
len(loanDF.toPandas()['title'].unique())

14424

In [16]:
loanDF.groupBy('STATE').agg(f.count('ID').alias('NumberofLessees')).toPandas()

,STATE,NumberofLessees
0,AZ,651
1,SC,341
2,LA,294
3,MN,435
4,NJ,1328
5,DC,152
6,OR,307
7,VA,998
8,None,1239
9,RI,149


In [17]:
loanDF.groupBy('term').agg(f.count('ID').alias('NumberofLessees')).toPandas()

,term,NumberofLessees
0,36 months,21706
1,60 months,8049


In [18]:
loanDF.groupBy('emp_length').agg(f.count('ID').alias('NumberofLessees')).toPandas()

,emp_length,NumberofLessees
0,9 years,940
1,5 years,2497
2,1 year,2440
3,n/a,792
4,2 years,3270
5,7 years,1332
6,8 years,1110
7,4 years,2591
8,6 years,1670
9,3 years,3062


In [19]:
loanDF.groupBy('home_ownership').agg(f.count('ID').alias('NumberofLessees')).toPandas()

,home_ownership,NumberofLessees
0,OWN,2272
1,RENT,14123
2,MORTGAGE,13288
3,OTHER,69
4,NONE,3


In [20]:
loanDF.groupBy('title').agg(f.count('ID').alias('NumberofLessees')).toPandas()

,title,NumberofLessees
0,Debit Free,1
1,Debt reducer,1
2,HOME REMODEL,3
3,My debt consolidation,2
4,Debt Consolidation 2011,13
5,Better rate,2
6,TC's,1
7,travel,3
8,Car Payoff,2
9,MY LOAN,7


### 2. Analizing null values, cross tables distribution and covariances

In [21]:
loanDF.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- loan_amnt: integer (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: string (nullable = true)
 |-- installment: double (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: double (nullable = true)
 |-- purpose: string (nullable = true)
 |-- title: string (nullable = true)
 |-- STATE: string (nullable = true)
 |-- delinq_2yrs: integer (nullable = true)
 |-- revol_bal: integer (nullable = true)
 |-- revol_util: string (nullable = true)
 |-- total_pymnt: double (nullable = true)
 |-- loan_status: integer (nullable = true)



In [22]:
loanDF.select([count(when(isnull(c), c)).alias(c) for c in loanDF.columns]).toPandas()

,ID,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,purpose,title,STATE,delinq_2yrs,revol_bal,revol_util,total_pymnt,loan_status
0,0,0,0,0,0,0,0,0,1229,1238,1239,1239,1239,1275,1239,0


In [23]:
nonullsDF = loanDF.where(col("revol_util").isNotNull())

In [24]:
nonullsDF.select([count(when(isnull(c), c)).alias(c) for c in nonullsDF.columns]).toPandas()

,ID,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,purpose,title,STATE,delinq_2yrs,revol_bal,revol_util,total_pymnt,loan_status
0,0,0,0,0,0,0,0,0,0,9,0,0,0,0,0,0


In [25]:
testDF.select([count(when(isnull(c),c)).alias(c) for c in testDF.columns]).toPandas()

,ID,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,purpose,title,STATE,delinq_2yrs,revol_bal,revol_util,total_pymnt,loan_status
0,0,0,0,0,0,286,0,0,392,392,394,394,394,406,394,0


In [26]:
col_names = nums
features = loanDF[nums].rdd.map(lambda row: row[0:])
corr_mat=Statistics.corr(features, method="pearson")
corr_df = pd.DataFrame(corr_mat)
corr_df.index, corr_df.columns = col_names, col_names
print(corr_df.to_string())

                   ID  loan_amnt  installment  annual_inc  delinq_2yrs  revol_bal  total_pymnt  loan_status
ID           1.000000  -0.143171    -0.080380   -0.005701          NaN        NaN          NaN    -0.029683
loan_amnt   -0.143171   1.000000     0.929723    0.259886          NaN        NaN          NaN     0.055284
installment -0.080380   0.929723     1.000000    0.258014          NaN        NaN          NaN     0.028359
annual_inc  -0.005701   0.259886     0.258014    1.000000          NaN        NaN          NaN    -0.037563
delinq_2yrs       NaN        NaN          NaN         NaN          1.0        NaN          NaN          NaN
revol_bal         NaN        NaN          NaN         NaN          NaN        1.0          NaN          NaN
total_pymnt       NaN        NaN          NaN         NaN          NaN        NaN          1.0          NaN
loan_status -0.029683   0.055284     0.028359   -0.037563          NaN        NaN          NaN     1.000000


In [27]:
for x in nums:
    for y in nums:
        covariance = loanDF.stat.cov(x,y)
        print(x + " and " + y + " covariance is: " + str(covariance))

ID and ID covariance is: 131382520.45297779
ID and loan_amnt covariance is: -12195810.527346931
ID and installment covariance is: -191484.43182982932
ID and annual_inc covariance is: -4357533.164519576
ID and delinq_2yrs covariance is: 41.774679564496
ID and revol_bal covariance is: -544089.9653014727
ID and total_pymnt covariance is: -14599251.569717444
ID and loan_status covariance is: -118.75775498447395
loan_amnt and ID covariance is: -12195810.52734693
loan_amnt and loan_amnt covariance is: 55229613.06533747
loan_amnt and installment covariance is: 1436013.2198559118
loan_amnt and annual_inc covariance is: 128790952.5486099
loan_amnt and delinq_2yrs covariance is: -111.91643190399427
loan_amnt and revol_bal covariance is: 35052983.98744884
loan_amnt and total_pymnt covariance is: 56278750.18753406
loan_amnt and loan_status covariance is: 143.40836146379573
installment and ID covariance is: -191484.4318298296
installment and loan_amnt covariance is: 1436013.2198559118
installment a

In [28]:
loanDF.stat.crosstab("loan_status", "term").toPandas()

,loan_status_term,36 months,60 months
0,1,2379,1846
1,0,19327,6203


In [29]:
loanDF.stat.crosstab("loan_status", "home_ownership").toPandas()

,loan_status_home_ownership,MORTGAGE,NONE,OTHER,OWN,RENT
0,1,1774,0,9,322,2120
1,0,11514,3,60,1950,12003


In [30]:
loanDF.stat.crosstab("loan_status", "emp_length").toPandas()

,loan_status_emp_length,1 year,10+ years,2 years,3 years,4 years,5 years,6 years,7 years,8 years,9 years,< 1 year,n/a
0,1,343,1006,427,387,346,348,235,205,160,120,478,170
1,0,2097,5599,2843,2675,2245,2149,1435,1127,950,820,2968,622


In [31]:
loanDF.stat.crosstab("loan_status", "purpose").toPandas()

,loan_status_purpose,car,credit_card,debt_consolidation,educational,home_improvement,house,major_purchase,medical,moving,null,other,renewable_energy,small_business,vacation,wedding
0,1,122,399,1991,39,249,45,166,75,66,174,447,13,334,41,64
1,0,999,3295,11414,197,1883,231,1420,431,346,1055,2423,55,953,230,598


In [32]:
loanDF.stat.crosstab("loan_status", "delinq_2yrs").toPandas()

,loan_status_delinq_2yrs,0,1,2,3,4,5,6,7,8,null
0,1,3545,376,92,24,9,2,1,1,1,174
1,0,21863,2023,388,133,37,13,5,2,1,1065


#### ETL summary

You should explain how you are going to clean and prepare the data.

Exploratory Data Analysis does not seem to find any blank or null numericals values. Entries of 'none' are found in some of the categorical variables. Furthermore, the "title" column has far too many unique values that must be reduced as all categoricals should be dummified. Categoricals otherwise do not appear to have too many invalid or dirty entries.

Interest rate and revolv_util will need to be converted to ints.

### 2.1 Spark code about the one explained in "ETL Summary"

In [33]:
#Reduce the number of categories for title column
# home_related = ['home','house','kitchen','bathroom','mortgage']
# loan_df = string_replace(loanDF,'title',home_related,'household')

# card_related = ['card','credit card','debit card']
# loan_df = string_replace(loanDF,'title',card_related,'card')

# auto_related = ['car','auto','automobile','truck']
# loan_df = string_replace(loanDF,'title',auto_related,'auto')

# wedding_related = ['wedding','marriage']
# loan_df = string_replace(loanDF,'title',wedding_related,'wedding')

# personal = ['personal','trip']
# loan_df = string_replace(loanDF,'title',personal,'personal')

cleanloanDF = nonullsDF.select([c for c in nonullsDF.columns if c not in {'title'}])
cleantestDF = testDF.select([c for c in testDF.columns if c not in {'title'}])

In [34]:
#Fix interest rates
cleanloanDF = cleanloanDF.withColumn('int_rate', f.regexp_replace('int_rate', '%', ''))
cleanloanDF = cleanloanDF.withColumn('int_rate', cleanloanDF['int_rate'].cast('float')/100)

cleantestDF = cleantestDF.withColumn('int_rate', f.regexp_replace('int_rate', '%', ''))
cleantestDF = cleantestDF.withColumn('int_rate', cleantestDF['int_rate'].cast('float')/100)

#Fix revolve_util
cleanloanDF = cleanloanDF.withColumn('revol_util', cleanloanDF['revol_util'].cast('float'))
cleanloanDF = cleanloanDF.where(col("revol_util").isNotNull())

cleantestDF = cleantestDF.withColumn('revol_util', cleantestDF['revol_util'].cast('float'))

cleanloanDF.limit(10).toPandas()

,ID,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,purpose,STATE,delinq_2yrs,revol_bal,revol_util,total_pymnt,loan_status
0,2,2500,60 months,0.1527,59.83,< 1 year,RENT,30000.0,car,GA,0,1687,0.094,1014.530000,1
1,4,10000,36 months,0.1349,339.31,10+ years,RENT,49200.0,other,CA,0,5598,0.210,12231.890000,0
2,5,3000,60 months,0.1269,67.79,1 year,RENT,80000.0,other,OR,0,27783,0.539,4066.908161,0
3,6,5000,36 months,0.0790,156.46,3 years,RENT,36000.0,wedding,AZ,0,7963,0.283,5632.210000,0
4,7,7000,60 months,0.1596,170.08,8 years,RENT,47004.0,debt_consolidation,NC,0,17726,0.856,10137.840010,0
5,8,3000,36 months,0.1864,109.43,9 years,RENT,48000.0,car,CA,0,8221,0.875,3939.135294,0
6,10,5375,60 months,0.1269,121.45,< 1 year,RENT,15000.0,other,TX,0,9279,0.365,1484.590000,1
7,11,6500,60 months,0.1465,153.45,5 years,OWN,72000.0,debt_consolidation,AZ,0,4032,0.206,7678.017673,0
8,12,12000,36 months,0.1269,402.54,10+ years,OWN,75000.0,debt_consolidation,CA,0,23336,0.671,13947.989160,0
9,14,3000,36 months,0.0991,96.68,3 years,RENT,15000.0,credit_card,IL,0,7323,0.431,3480.269999,0


In [35]:
categoricals = categorical_features(cleanloanDF)
categoricals.append('delinq_2yrs')
categoricals.append('loan_status')

stages = []

### 2.2 Spark code about the transformations you apply to the variables

### String to Index

In [36]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer

indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(cleanloanDF) for column in categoricals ]

for n in range(len(indexers)):
    stages.append(indexers[n])


pipeline = Pipeline(stages=indexers)
df_r = pipeline.fit(cleanloanDF).transform(cleanloanDF)

df_r.limit(10).toPandas()

,ID,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,purpose,STATE,...,revol_util,total_pymnt,loan_status,term_index,emp_length_index,home_ownership_index,purpose_index,STATE_index,delinq_2yrs_index,loan_status_index
0,2,2500,60 months,0.1527,59.83,< 1 year,RENT,30000.0,car,GA,...,0.094,1014.530000,1,1.0,1.0,0.0,6.0,7.0,0.0,1.0
1,4,10000,36 months,0.1349,339.31,10+ years,RENT,49200.0,other,CA,...,0.210,12231.890000,0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
2,5,3000,60 months,0.1269,67.79,1 year,RENT,80000.0,other,OR,...,0.539,4066.908161,0,1.0,6.0,0.0,2.0,24.0,0.0,0.0
3,6,5000,36 months,0.0790,156.46,3 years,RENT,36000.0,wedding,AZ,...,0.283,5632.210000,0,0.0,3.0,0.0,7.0,12.0,0.0,0.0
4,7,7000,60 months,0.1596,170.08,8 years,RENT,47004.0,debt_consolidation,NC,...,0.856,10137.840010,0,1.0,9.0,0.0,0.0,15.0,0.0,0.0
5,8,3000,36 months,0.1864,109.43,9 years,RENT,48000.0,car,CA,...,0.875,3939.135294,0,0.0,10.0,0.0,6.0,0.0,0.0,0.0
6,10,5375,60 months,0.1269,121.45,< 1 year,RENT,15000.0,other,TX,...,0.365,1484.590000,1,1.0,1.0,0.0,2.0,3.0,0.0,1.0
7,11,6500,60 months,0.1465,153.45,5 years,OWN,72000.0,debt_consolidation,AZ,...,0.206,7678.017673,0,1.0,5.0,2.0,0.0,12.0,0.0,0.0
8,12,12000,36 months,0.1269,402.54,10+ years,OWN,75000.0,debt_consolidation,CA,...,0.671,13947.989160,0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
9,14,3000,36 months,0.0991,96.68,3 years,RENT,15000.0,credit_card,IL,...,0.431,3480.269999,0,0.0,3.0,0.0,1.0,5.0,0.0,0.0


### OneHot Encode

In [37]:
from pyspark.ml.feature import OneHotEncoderEstimator
encoder = OneHotEncoderEstimator(inputCols=['term_index', 'emp_length_index', 'home_ownership_index', 'purpose_index', 'STATE_index', 'delinq_2yrs_index', 'loan_status_index'],
                                 outputCols=["term_vec", "emp_length_vec", "home_ownership_vec", "purpose_vec", "STATE_vec", "delinq_2yrs_vec", "loan_status_vec"])

stages.append(encoder)

model = encoder.fit(df_r)
encoded = model.transform(df_r)
encoded.limit(10).toPandas()



,ID,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,purpose,STATE,...,STATE_index,delinq_2yrs_index,loan_status_index,purpose_vec,STATE_vec,loan_status_vec,home_ownership_vec,delinq_2yrs_vec,term_vec,emp_length_vec
0,2,2500,60 months,0.1527,59.83,< 1 year,RENT,30000.0,car,GA,...,7.0,0.0,1.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...",(0.0),"(1.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",(0.0),"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,4,10000,36 months,0.1349,339.31,10+ years,RENT,49200.0,other,CA,...,0.0,0.0,0.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",(1.0),"(1.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",(1.0),"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,5,3000,60 months,0.1269,67.79,1 year,RENT,80000.0,other,OR,...,24.0,0.0,0.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",(1.0),"(1.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",(0.0),"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."
3,6,5000,36 months,0.0790,156.46,3 years,RENT,36000.0,wedding,AZ,...,12.0,0.0,0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",(1.0),"(1.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",(1.0),"(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,7,7000,60 months,0.1596,170.08,8 years,RENT,47004.0,debt_consolidation,NC,...,15.0,0.0,0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",(1.0),"(1.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",(0.0),"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
5,8,3000,36 months,0.1864,109.43,9 years,RENT,48000.0,car,CA,...,0.0,0.0,0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",(1.0),"(1.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",(1.0),"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
6,10,5375,60 months,0.1269,121.45,< 1 year,RENT,15000.0,other,TX,...,3.0,0.0,1.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",(0.0),"(1.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",(0.0),"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
7,11,6500,60 months,0.1465,153.45,5 years,OWN,72000.0,debt_consolidation,AZ,...,12.0,0.0,0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",(1.0),"(0.0, 0.0, 1.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",(0.0),"(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
8,12,12000,36 months,0.1269,402.54,10+ years,OWN,75000.0,debt_consolidation,CA,...,0.0,0.0,0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",(1.0),"(0.0, 0.0, 1.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",(1.0),"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
9,14,3000,36 months,0.0991,96.68,3 years,RENT,15000.0,credit_card,IL,...,5.0,0.0,0.0,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...",(1.0),"(1.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",(1.0),"(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


### 2.3 Code to assemble the variables to a numerical vector (VectorAssembler)

### Assemble Vectors

In [38]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=["loan_amnt", 'int_rate', 'installment', 'annual_inc', 'revol_bal', 'revol_util', 'total_pymnt', 'purpose_vec', 'STATE_vec', 'home_ownership_vec', 'delinq_2yrs_vec', 'term_vec', 'emp_length_vec'],
    outputCol="features")

stages.append(assembler)

output = assembler.transform(encoded)

output.limit(10).toPandas()

,ID,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,purpose,STATE,...,delinq_2yrs_index,loan_status_index,purpose_vec,STATE_vec,loan_status_vec,home_ownership_vec,delinq_2yrs_vec,term_vec,emp_length_vec,features
0,2,2500,60 months,0.1527,59.83,< 1 year,RENT,30000.0,car,GA,...,0.0,1.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...",(0.0),"(1.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",(0.0),"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(2500.0, 0.15270000457763672, 59.83, 30000.0, ..."
1,4,10000,36 months,0.1349,339.31,10+ years,RENT,49200.0,other,CA,...,0.0,0.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",(1.0),"(1.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",(1.0),"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(10000.0, 0.13489999771118164, 339.31, 49200.0..."
2,5,3000,60 months,0.1269,67.79,1 year,RENT,80000.0,other,OR,...,0.0,0.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",(1.0),"(1.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",(0.0),"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","(3000.0, 0.12689999580383302, 67.79, 80000.0, ..."
3,6,5000,36 months,0.0790,156.46,3 years,RENT,36000.0,wedding,AZ,...,0.0,0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",(1.0),"(1.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",(1.0),"(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(5000.0, 0.07900000095367432, 156.46, 36000.0,..."
4,7,7000,60 months,0.1596,170.08,8 years,RENT,47004.0,debt_consolidation,NC,...,0.0,0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",(1.0),"(1.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",(0.0),"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(7000.0, 0.15960000038146974, 170.08, 47004.0,..."
5,8,3000,36 months,0.1864,109.43,9 years,RENT,48000.0,car,CA,...,0.0,0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",(1.0),"(1.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",(1.0),"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(3000.0, 0.18639999389648437, 109.43, 48000.0,..."
6,10,5375,60 months,0.1269,121.45,< 1 year,RENT,15000.0,other,TX,...,0.0,1.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",(0.0),"(1.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",(0.0),"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(5375.0, 0.12689999580383302, 121.45, 15000.0,..."
7,11,6500,60 months,0.1465,153.45,5 years,OWN,72000.0,debt_consolidation,AZ,...,0.0,0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",(1.0),"(0.0, 0.0, 1.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",(0.0),"(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","(6500.0, 0.14649999618530274, 153.45, 72000.0,..."
8,12,12000,36 months,0.1269,402.54,10+ years,OWN,75000.0,debt_consolidation,CA,...,0.0,0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",(1.0),"(0.0, 0.0, 1.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",(1.0),"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(12000.0, 0.12689999580383302, 402.54, 75000.0..."
9,14,3000,36 months,0.0991,96.68,3 years,RENT,15000.0,credit_card,IL,...,0.0,0.0,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...",(1.0),"(1.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",(1.0),"(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(3000.0, 0.0990999984741211, 96.68, 15000.0, 7..."


### 2.4 Create Pipeline for train and test data

In [39]:
pipeline = Pipeline(stages=stages)


In [ ]:
stages

In [40]:
train = pipeline.fit(cleanloanDF).transform(cleanloanDF)
test = pipeline.fit(cleantestDF).transform(cleantestDF)


In [41]:
train.describe().toPandas()

,summary,ID,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,purpose,...,revol_util,total_pymnt,loan_status,term_index,emp_length_index,home_ownership_index,purpose_index,STATE_index,delinq_2yrs_index,loan_status_index
0,count,27782,27782,27782,27782,27782,27782,27782,27782,27782,...,27782,27782,27782,27782,27782,27782,27782,27782,27782,27782
1,mean,19885.97379598301,11184.184004031387,None,0.12045145358596483,323.7282657835985,None,None,68968.50356525806,None,...,0.5008419840093695,12191.76996548699,0.1419624217118998,0.26869915772802533,3.5944496436541646,0.607119717802894,1.9604060182852205,8.781117270174933,0.14333021380750127,0.1419624217118998
2,stddev,11525.726776825126,7406.266658924296,None,0.037185925709493105,207.44610343924032,None,None,67607.99035948986,None,...,0.2768659597752423,9090.091277426953,0.3490178751186077,0.4432910934599531,3.159200874436869,0.637848279188638,2.7655106046897013,9.50419490649801,0.47889322807729845,0.3490178751186077
3,min,2,500,36 months,0.05420000076293945,16.08,1 year,MORTGAGE,4000.0,car,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,max,39759,35000,60 months,0.2459000015258789,1305.19,n/a,RENT,6000000.0,wedding,...,0.999,58886.47343,1,1.0,11.0,4.0,13.0,49.0,8.0,1.0


### 3. Logistic Regresion Model

### 3.1 Write a function "metrics" which has a LogisticRegressionModel.summary as input attribute and produces an output of: 
1. Area under ROC
2. False Positive Rate By Label
3. True Positive Rate By Label
4. Precision By Label
5. Recall By Label
6. fMeasure By Label
7. Accuracy
8. False Positive Rate
9. True Positive Rate
10. fMeasure
11. Precision
12. Recall

In [42]:
def metrics(trainingSummary):
    print("areaUnderROC: " + str(trainingSummary.areaUnderROC))
    print("False Positive Rate By Label: " + str(trainingSummary.falsePositiveRateByLabel))
    print("True Positive Rate By Label: " + str(trainingSummary.truePositiveRateByLabel))
    print("Precision By Label: " + str(trainingSummary.precisionByLabel))
    print("Recall By Label: " + str(trainingSummary.recallByLabel))
    print("fMeasure By Label: " + str(trainingSummary.fMeasureByLabel))
    print("Accuracy: " + str(trainingSummary.accuracy))
    print("False Positive Rate: " + str(trainingSummary.weightedFalsePositiveRate))
    print("True Positive Rate: " + str(trainingSummary.weightedTruePositiveRate))
    print("fMeasure: " + str(trainingSummary.weightedFMeasure))
    print("Precision: " + str(trainingSummary.weightedPrecision))
    print("Recall: " + str(trainingSummary.weightedRecall))
    

    
    
    



    
    

### 3.2 Apply a Logistic Regresion Base Model and show the metrics by the function above

In [43]:
from pyspark.ml.classification import LogisticRegression

lrm = LogisticRegression().setFeaturesCol('features').setLabelCol('loan_status')


# Fit the model
lrModel = lrm.fit(train)





In [44]:
trainingSummary = lrModel.summary

metrics(trainingSummary)


areaUnderROC: 0.9484985663029374
False Positive Rate By Label: [0.27560851926977686, 0.008599714741169561]
True Positive Rate By Label: [0.9914002852588304, 0.7243914807302231]
Precision By Label: [0.9560275080906149, 0.9330502939255388]
Recall By Label: [0.9914002852588304, 0.7243914807302231]
fMeasure By Label: <bound method LogisticRegressionSummary.fMeasureByLabel of <pyspark.ml.classification.BinaryLogisticRegressionTrainingSummary object at 0x000001941C9F45F8>>
Accuracy: 0.95349506874955
False Positive Rate: 0.23770330276049648
True Positive Rate: 0.95349506874955
fMeasure: <bound method LogisticRegressionSummary.weightedFMeasure of <pyspark.ml.classification.BinaryLogisticRegressionTrainingSummary object at 0x000001941C9F45F8>>
Precision: 0.9527656071235477
Recall: 0.95349506874955


### 3.2 We are going to try to improve our model:
1. Using a `weight column` in our Logistic Regression Model (Take into account we are working with a unbalanced dataset)
2. Define a `ParamGridBuilder` with `regParam`, `elasticNetParam` and `maxIter` at least
3. Define an `BinaryClassificationEvaluator`
4. Using Cross Validation with a 5-fold `CrossValidator`

Questions to answer:
1. Have we improved the ROC-AUC?
2. Which are the average ROC-AUC measurements in the different cross validation runs?
3. Which are the parameters of the best model in the 5 k-fold runs?
4. Which are the metrics of the best model (training) in the 5 k-fold runs? (Use the function above)
5. Which is the ROC-AUC on validation dataset?


#### 1. Balancing the dataset

In [45]:
ratio = train.select(avg("loan_status")).collect()[0]['avg(loan_status)']

In [46]:
train=train.withColumn("classWeights", when(train.loan_status == 1,ratio).otherwise(1-ratio))
train.limit(5).toPandas()

,ID,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,purpose,STATE,...,loan_status_index,purpose_vec,STATE_vec,loan_status_vec,home_ownership_vec,delinq_2yrs_vec,term_vec,emp_length_vec,features,classWeights
0,2,2500,60 months,0.1527,59.83,< 1 year,RENT,30000.0,car,GA,...,1.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...",(0.0),"(1.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",(0.0),"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(2500.0, 0.15270000457763672, 59.83, 30000.0, ...",0.141962
1,4,10000,36 months,0.1349,339.31,10+ years,RENT,49200.0,other,CA,...,0.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",(1.0),"(1.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",(1.0),"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(10000.0, 0.13489999771118164, 339.31, 49200.0...",0.858038
2,5,3000,60 months,0.1269,67.79,1 year,RENT,80000.0,other,OR,...,0.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",(1.0),"(1.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",(0.0),"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","(3000.0, 0.12689999580383302, 67.79, 80000.0, ...",0.858038
3,6,5000,36 months,0.0790,156.46,3 years,RENT,36000.0,wedding,AZ,...,0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",(1.0),"(1.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",(1.0),"(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(5000.0, 0.07900000095367432, 156.46, 36000.0,...",0.858038
4,7,7000,60 months,0.1596,170.08,8 years,RENT,47004.0,debt_consolidation,NC,...,0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",(1.0),"(1.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",(0.0),"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(7000.0, 0.15960000038146974, 170.08, 47004.0,...",0.858038


In [47]:
lrm = LogisticRegression().setFeaturesCol('features').setLabelCol('loan_status').setWeightCol('classWeights')

In [48]:
lrModel = lrm.fit(train)
trainingSummary = lrModel.summary

metrics(trainingSummary)

areaUnderROC: 0.9448218191691826
False Positive Rate By Label: [0.36359026369168357, 0.0009228962161255138]
True Positive Rate By Label: [0.9990771037838745, 0.6364097363083164]
Precision By Label: [0.9432079207920792, 0.9913112164296999]
Recall By Label: [0.9990771037838745, 0.6364097363083164]
fMeasure By Label: <bound method LogisticRegressionSummary.fMeasureByLabel of <pyspark.ml.classification.BinaryLogisticRegressionTrainingSummary object at 0x000001941C8642B0>>
Accuracy: 0.9475919660211648
False Positive Rate: 0.3121051259289738
True Positive Rate: 0.9475919660211648
fMeasure: <bound method LogisticRegressionSummary.weightedFMeasure of <pyspark.ml.classification.BinaryLogisticRegressionTrainingSummary object at 0x000001941C8642B0>>
Precision: 0.9500367811331192
Recall: 0.9475919660211648


In [49]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
paramGrid = ParamGridBuilder()\
    .addGrid(lrm.elasticNetParam,[0.0, 0.5, 1.0])\
    .addGrid(lrm.fitIntercept,[False, True])\
    .addGrid(lrm.maxIter,[10, 100])\
    .addGrid(lrm.regParam,[0.01, 0.5, 2.0]) \
    .build()

In [50]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator=BinaryClassificationEvaluator(rawPredictionCol='rawPrediction',labelCol='loan_status')


In [51]:
cv = CrossValidator() \
        .setEstimator(lrm) \
        .setEvaluator(evaluator) \
        .setEstimatorParamMaps(paramGrid) \
        .setNumFolds(5)

In [55]:
(trainingData, validationData) = train.randomSplit([0.8, 0.2], seed=100)
print(trainingData.count())
print(validationData.count())

22255
5527


In [56]:
cvModel = cv.fit(trainingData)




In [66]:
cv_results = cvModel.transform(validationData)

# Before we had 0.9024 and now
print("AUC: " + str(evaluator.evaluate(cv_results)))

# Means of model accuracy
print("Means of metrics: " + str(cvModel.avgMetrics))

AUC: 0.903740357368801
Means of metrics: [0.8567073812528986, 0.49500631377317394, 0.4719855677607401, 0.8934624459055573, 0.49500631377317394, 0.4719855677607401, 0.8939606106668602, 0.7865651297493514, 0.7816079020932529, 0.8929840932049582, 0.7865029754029198, 0.781674191895706, 0.7152845910861866, 0.3663788711684737, 0.34593261008680243, 0.8571962301959661, 0.36645813660460913, 0.34593261008680243, 0.9016532794779921, 0.5, 0.5, 0.9059017344765015, 0.5, 0.5, 0.7273985313616557, 0.34941508837321167, 0.5, 0.7958255644195149, 0.3463371750780666, 0.5, 0.8168598371715454, 0.5, 0.5, 0.8821523909047524, 0.5, 0.5]


### 4. Random Forest Model
1. Define a `ParamGridBuilder` with `maxDepth`, `numTrees` and `maxIter` at least
2. Define an `BinaryClassificationEvaluator` (You can use the above one)
3. Using Cross Validation with a 5-fold `CrossValidator`

Questions to answer:

1. Have we improved the ROC-AUC?
2. Which are the average ROC-AUC measurements in the different cross validation runs?
3. Which are the parameters of the best model in the 5 k-fold runs?
4. Which is the importance of the features?
5. Print full description of model.
6. Which is the ROC-AUC on validation dataset?

In [92]:
from pyspark.ml.classification import RandomForestClassificationModel, RandomForestClassifier

rf = RandomForestClassifier() \
        .setFeaturesCol("features") \
        .setLabelCol("loan_status") \
        .setSeed(100)

paramGrid = ParamGridBuilder() \
                .addGrid(rf.maxDepth, [10,30]) \
                .addGrid(rf.numTrees, [10,100]) \
                .addGrid(rf.maxBins,[16,32,64])\
                .build()

cv = CrossValidator() \
        .setEstimator(rf) \
        .setEvaluator(evaluator) \
        .setEstimatorParamMaps(paramGrid) \
        .setNumFolds(5)

In [ ]:
cvModel = cv.fit(trainingData)

### 5. Gradient Boosting Model
1. Defining a `ParamGridBuilder` with `maxDepth`, `numTrees` and `maxIter` at least (You can use the above one)
2. Define an `BinaryClassificationEvaluator` (You can use the above one)
3. Using Cross Validation with a 5-fold `CrossValidator`

Questions to answer:

1. Have we improved the ROC-AUC?
2. Which are the average ROC-AUC measurements in the different cross validation runs?
3. Which are the parameters of the best model in the 5 k-fold runs?
4. Which is the importance of the features?
5. Print full description of model.
6. Which is the ROC-AUC on validation dataset?

In [92]:
from pyspark.ml.classification import GBTClassifier

gbt = GBTClassifier() \
        .setFeaturesCol("features") \
        .setLabelCol("loan_status") \
        .setSeed(100)

paramGrid = ParamGridBuilder() \
                .addGrid(gbt.maxDepth, [10,30]) \
                .addGrid(gbt.numTrees, [10,100]) \
                .addGrid(gbt.maxBins,[16,32,64])\
                .addGrid(gbt.maxIter, [10,100])
                .build()

cv = CrossValidator() \
        .setEstimator(gbt) \
        .setEvaluator(evaluator) \
        .setEstimatorParamMaps(paramGrid) \
        .setNumFolds(5)

### 6. Apply your best model to send the predictions on test

In [ ]:
sc.stop()